https://github.com/jakevdp/PythonDataScienceHandbook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as sps
import matplotlib.pylab as plt

In [ ]:
def my_basic_research(df, column = "volume"):
    '''
        Функция выводит описание метрики,  
        самые популярные значения метрики  
        и распределение метрики
    '''
    print("Базовые метрики")
    print(df[column].describe())
    print("------------------------------------")
    
    print("Самые популярные значения метрики, топ 5")
    print(df[column].value_counts().nlargest(5))
    print("------------------------------------")
    
    print("Эксцесс ", sps.kurtosis(df[column]))
    print("Ассиметрия ", sps.skew(df[column]))
    
    sns.distplot(df[column])
    plt.title("Распределение прибыли по пользователям")

Следующий код проверяет на какое распределение похожа выборка из заданных распределений.  
Параметр param_bins нужно поставить таким, чтобы его значение было не больше размера выборки.   
https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python  

In [ ]:
import scipy.stats as st
import matplotlib
import warnings

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

param_bins = 100

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

# Load data from statsmodels datasets
data = df_norm['Empty_Index']

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=param_bins, normed=True, alpha=0.5, color=list(plt.rcParams['axes.prop_cycle'])[1]['color'])
            #   plt.rcParams['axes.prop_cycle'][1])
# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_name, best_fit_params = best_fit_distribution(data, param_bins, ax)
best_dist = getattr(st, best_fit_name)

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Pixels_Areas\n All Fitted Distributions')
ax.set_xlabel(u'Pixels_areas')
ax.set_ylabel('Frequency')

# Make PDF with best params 
pdf = make_pdf(best_dist, best_fit_params)

# Display
plt.figure(figsize=(12,8))
ax = pdf.plot(lw=2, label='PDF', legend=True)
data.plot(kind='hist', bins=param_bins, normed=True, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
dist_str = '{}({})'.format(best_fit_name, param_str)

ax.set_title(u'Pixels_Areas \n Best distribution is ' + dist_str)
ax.set_xlabel(u'Pixels_Areas')
ax.set_ylabel('Frequency')

Функция строить линейный график, когда по оси X у нас дата

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def lineplot_date(data, date_col, y_col, color, graf_name):
    '''
    Функция строит график, когда по оси дата
    data - датафрейм с данными
    date_col - колонка с датой
    y_col - колонка для оси y
    color - цвет линии графика
    graf_name - название графика
    '''
    plt.figure(figsize=(15, 5))
    sns.set_style('whitegrid', {'axes.facecolor': '1'
                                , 'grid.color': '0.9'
                                , 'figure.facecolor': '1.0'})
    sns.lineplot(x=date_col
                 , y=float(y_col.mean())
                 , data=data
                 , color=color
                 , alpha=.5) # линия среднего значения
    plt.text(data[date_col].min
             , float(data.mean())*1.03
             , ' среднее значение = ' + str(round(data.mean(), 2))
             , color=color
             , ha='left')
    ms = 10 if len(data[date_col]) < 60 else 0 # задаем наличие и размер маркера в зависимоти от плотности графика
    sns.lineplot(x=date_col, y=y_col, data=data, color=color, linewidth=2.2, marker='o', markersize=ms)
    plt.xlim(data[date_col].min(), data[date_col].max())
    plt.ylim(data[y_col].min(), data[y_col].max()*1.1)
    plt.xticks(alpha=.5)
    plt.yticks(alpha=.5)
    plt.xlabel('', alpha=.5)
    plt.ylabel('', alpha=.5)
    plt.title(graf_name, fontsize=14, pad=15)
    plt.gca().spines['top'].set_alpha(0.3)
    plt.gca().spines['left'].set_alpha(0.3)
    plt.gca().spines['right'].set_alpha(0.3)
    plt.gca().spines['bottom'].set_alpha(0.3)
    

Удобная функция для построения графиков

In [ ]:
def plot(x, y=None, title=None, x_label=None, y_label=None, kind='plot', **kwargs):
    f, n = {
        'plot': (plt.plot, 2),
        'line': (plt.plot, 2),
        'scatter': (plt.scatter, 2),
        'hist': (plt.hist, 1),
        'dist': (sns.distplot, 1),
        'box': (sns.boxplot, 1),
        'violin': (sns.violinplot, 1),
        'reg': (sns.regplot, 2),
        'lm': (sns.lmplot, 2),
    }[kind]
    f(x, **kwargs) if n == 1 else f(x, y, **kwargs)
    ax = kwargs['ax'] if 'ax' in kwargs else plt.gca()
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    return ax

Модернизация pandas info

In [ ]:
# переделать так, чтобы инфо выводилось измененное
# убрать оттуда количество не нулевых  
# заменить их на количество пропусков и процент пропусков
# сделать из этого дата фрейм и выводить под smaple
# можно добавить ещё полезной инфы
def info(data):
    display(data.sample(5, random_state=2))
    display(data.info())
    display(pd.DataFrame(round((df.isna().mean()*100),2)).style.background_gradient('coolwarm'))

Замена значений в столбце в зависимости от категории  
То есть например мы заменяем не на медиану по всему дата фрейму, а только внутри определенной группы

In [ ]:
df_temp['total_income'] = df_temp.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
# изменить функцию с учетом строки выше
def no_non_value(df, col_with_null, category):
    df_in = df_in.copy()
    for type_unique in df_in[category].unique():
        df_in.loc[(df_in[col_with_null].isna())&(df_in[category] ==type_unique), col_with_null] = \
        df_in.loc[df_in[category] ==type_unique, col_with_null].median()
    return df_in

Можно делать через коэффициент

In [ ]:
df['liv_coef'] = df['living_area']/df['total_area'] #создадим коэффициенты, чтоб уточнить данные, 
df['kit_coef'] = df['kitchen_area']/df['total_area']#разделив площади (жилые и кухни) на общую

In [ ]:
# а тут просто сократим код, исключить ненужные переменные
df['living_area'] = df['living_area'].fillna((df['rooms'].map(df.groupby(
    'rooms')['liv_coef'].median().to_dict()))*df['total_area'])

df['kitchen_area'] = df['kitchen_area'].fillna((df['rooms'].map(df.groupby(
    'rooms')['kit_coef'].median().to_dict()))*df['total_area'])

df['kitchen_area'] = df['kitchen_area'].fillna(0)
df['days_exposition'] = df['days_exposition'].fillna(0)

Использование лематизации для категоризации 

In [ ]:
from pymystem3 import Mystem # импортируем библиотеку для лемматизации
m = Mystem()

def lemmatize_purpose(string):# создаем функцию для лемматизации отдельной строки
    lemma = m.lemmatize(string)
    return lemma

df['purpose'] = df['purpose'].apply(lemmatize_purpose)

dest_str = ['жилье', 'автомобиль', 'образование', 'недвижимость', 'свадьба']

def filter_purpose(srt_lem):
    """
    создаем функцию, которая отсеивает ненужные слова, возвращая только основные понятия
    """
    for filtr in dest_str:       # перебераем с помощью переменной filtr каждое понятие
        if filtr in srt_lem:     # сравниваем со строкой из таблицы
            return filtr         # в случае успеха функция возвращает основное понятие
                
df['purpose'] = df['purpose'].apply(filter_purpose)

Функция для определения числа бинов для гистограмы

In [ ]:
def my_bins(df,value):
    '''
    Сделаем функцию которая отобразит нужное количество корзин 
    '''    
    if df[value].max() > 100:
        return 50
    elif df[value].max() > 500:
        return  100
    elif df[value].max() > 1000:
        return  200
    else: return 20

In [ ]:
def pivot_math_index(data, datetime, index, values, func):
    """
    Аргументы функции: data - изначальная таблица
    datetime - столбец с дата-временем из которого функция вычислит месяц
    index - индексы в будущей сводной таблице
    values - значения с которыми будут производится вычисление
    func - функция, производящее вычисление
    
    Функция возвращает сводную таблицу в которой просчитывает значения по месяцам, 
    для каждого пользователя
    """
    data[datetime] = pd.to_datetime(data[datetime], format = ('%Y-%m-%d'))
    data['month'] = data[datetime].dt.month
    pivot = data.pivot_table(index = [index,'month'], values = values, aggfunc = func, fill_value = 0)
    #pivot=pivot.reset_index()
    return pivot

In [ ]:
### Бонус, функция для построения визуализаций к третьему проекту)
def mean_rel_dev_hist(data, data2):
    tariff_list=['"Смарт"','"Ультра"']
    mn = [
        'December', 
        'November', 
        'October', 
        'September', 
        'August', 
        'July', 
        'June', 
        'May', 
        'April', 
        'March', 
        'February', 
        'January'
    ]
    plot_list = ['mb_count', 'messages_count', 'calls_min']
    colors_list = [['#86bf91', 'orange'], ['purple', 'turquoise'], ['red', 'orange']]
    title_list = [
        'Среднее количество Mb трафика в месяц', 
        'Среднее количество сообщений в месяц', 
        'Среднее кол-во минут разговора в месяц'
    ]
    title_list_hist = [
        'Распределение Mb трафика по пользователям', 
        'Распределение количества сообщений по пользователям', 
        'Распределение кол-ва минут разговоров по пользователям'
    ]
    border_list = [[15360, 30720], [50, 1000], [500, 3000]]
    df_list = [data, data2]
    hight = [-0.07, 0.3]
       
    for i in range(len(plot_list)):
        fig, axes = plt.subplots(figsize=(15, 8))
        fig1, axes1 = plt.subplots(figsize=(15.7, 6))
        
        for j in range(len(df_list)):
            
            df = df_list[j].pivot_table(index='month', values=plot_list[i])
            df = df.reindex(mn)
            df[plot_list[i]] = df[plot_list[i]].astype(int)
            expenses_dict = dict(df[plot_list[i]])
            label = (
                '\nСреднее - {:.2f}\nДисперсия - {:.2f}\nstd - {:.2f}'
                .format(df_list[j][plot_list[i]].mean(), 
                        df_list[j][plot_list[i]].var(ddof=1), 
                        df_list[j][plot_list[i]].std(ddof=1)
                       )
            )
            plot = df.plot(
                y=plot_list[i],
                kind='barh', 
                ec='black', 
                ax=axes, 
                width=0.4, 
                position = j, 
                title=title_list[i], 
                color=colors_list[i][j], 
                label='Тариф ' + tariff_list[j]
            )
            h=hight[j]
            k = 0
            axes.set_ylim(ymin=-0.5, ymax=11.5)
            for key in expenses_dict:            
                if i == 0:
                    mb = str(expenses_dict[key]) + ' mb'
                    plot.text(
                        expenses_dict[key]- 1880, 
                        k-h, 
                        mb, 
                        color = 'white', 
                        fontsize='large', 
                        fontweight='bold'
                    )
                elif i == 1:
                    plot.text(
                        expenses_dict[key] - 1.5, 
                        k-h, 
                        expenses_dict[key], 
                        color = 'white', 
                        fontsize='large', 
                        fontweight='bold'
                    )
                else:
                    min = str(expenses_dict[key]) + ' min'
                    plot.text(
                        expenses_dict[key] - 45, 
                        k-h, 
                        min, 
                        color = 'white', 
                        fontsize='large', 
                        fontweight='bold'
                    )
                k += 1
            hist = df_list[j][plot_list[i]].plot(
                kind='hist', 
                bins=40, 
                ax=axes1, 
                alpha=0.5, 
                ec='black', 
                color = colors_list[i][j], 
                label='Тариф ' + tariff_list[j] + label, 
                title = title_list_hist[j]
            )
            plt.axvline(
                x=df_list[j][plot_list[i]].median(),
                linewidth=3, 
                color=colors_list[i][j], 
                linestyle='--', 
                label='Медиана ' + tariff_list[j]
            )
            plt.axvline(
                x=df_list[j][plot_list[i]].mean(),
                linewidth=3, 
                color=colors_list[i][j], 
                linestyle='dotted', 
                label='Среднее ' + tariff_list[j]
            )
            plt.axvline(
                x=border_list[i][j], 
                linewidth=4, 
                color=colors_list[i][j], 
                linestyle='solid', 
                label='Лимит ' + tariff_list[j]
            )
            axes1.set_xlim(xmin=0)
            plt.legend(loc='upper right')